# Book Recommender System

In [1]:
!pip install dgl
import dgl
import torch
import pandas as pd
import numpy as np

In [349]:
ratings = pd.read_csv("Data/Ratings.csv")
print(f"Ratings Data Shape: {ratings.shape} \n")
print(f"{ratings.dtypes}\n")
print(ratings.head())

Ratings Data Shape: (1149780, 3) 

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object

   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


In [390]:
books = pd.read_csv("Data/Books.csv")
print(f"Books Data Shape: {books.shape} \n")
print(f"{books.dtypes}\n")
print(books.head(3))

Books Data Shape: (271360, 8) 

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object

         ISBN            Book-Title           Book-Author Year-Of-Publication  \
0  0195153448   Classical Mythology    Mark P. O. Morford                2002   
1  0002005018          Clara Callan  Richard Bruce Wright                2001   
2  0060973129  Decision in Normandy          Carlo D'Este                1991   

                 Publisher                                        Image-URL-S  \
0  Oxford University Press  http://images.amazon.com/images/P/0195153448.0...   
1    HarperFlamingo Canada  http://images.amazon.com/images/P/0002005018.0...   
2          HarperPerennial  http://images.amazon.com/images/P/0060973129.0...   

                                         Image-URL-M  \
0  http

/var/folders/qc/8fy08h5s33n4rrmz09_8bv680000gn/T/ipykernel_12836/3035873185.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Data/Books.csv")


In [393]:
print('Nan-values by column')
print(books.isna().sum())

Nan-values by column
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [389]:
users = pd.read_csv("Data/Users.csv")
print(f"Users Data Shape: {users.shape} \n")
print(f"{users.dtypes}\n")
print(users.head())

print('Nan-values by column')
print(users.isna().sum())

Users Data Shape: (278858, 3) 

User-ID       int64
Location     object
Age         float64
dtype: object

   User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN
Nan-values by column
User-ID          0
Location         0
Age         110762
dtype: int64


## Preprocessing

In [406]:
# We will only use users and books present in the ratings dataset 
lessen_user_ids = {userid: idx for idx, userid in enumerate(ratings['User-ID'].unique())} #renumber IDs to reduce inactive users
ratings['New-User-ID'] = ratings['User-ID'].map(lessen_user_ids)
user_ids = list(ratings['New-User-ID'].unique())
num_users = len(set(user_ids))

# Map book identifiers (ISBN) to a unique integer identifier for datatype compatibility of dgl
isbn_to_id = {isbn: idx for idx, isbn in enumerate(ratings['ISBN'].unique())}
ratings['Book-ID'] = ratings['ISBN'].map(isbn_to_id)
book_ids = list(ratings['Book-ID'].unique())
num_books = len(set(book_ids))

print(f'There are {len(user_ids)} unique users, and {len(book_ids)} unique books in the ratings dataset.')
 
# Remove users and books not included in the ratings dataset
books['Book-ID'] = books['ISBN'].map(isbn_to_id)
books_clean = books[books['Book-ID'].isin(book_ids)]
books_clean_ids = books_clean['Book-ID'].unique()
percent_books_missing = (num_books-len(books_clean_ids))/num_books*100

print(f'There are {percent_books_missing}% of books in the graph missing in the books data')

users['New-User-ID'] = users['User-ID'].map(lessen_user_ids)
users_clean = users[users['New-User-ID'].isin(user_ids)]

There are 105283 unique users, and 340556 unique books in the ratings dataset.
There are 20.673545613643572% of books in the graph missing in the books data


1/5 of the books that have rating information do not have further information on the books dataset. However, as our objective is to investivate an user-based recommender system, this is irrelevant. We are able to embed the age and location data of users. As the age data is sparse, location data will be our main source of information.

In [ ]:
# Make graph
src = torch.tensor(ratings['New-User-ID'].values)
dst = torch.tensor(ratings['Book-ID-offset'].values)

edges = {
    ('user', 'rating', 'book'): (src, dst)
}

g = dgl.heterograph(edges, num_nodes_dict={'user': num_users, 'book': num_books})
print(g)

Graph(num_nodes={'book': 340556, 'user': 105283},
      num_edges={('user', 'rating', 'book'): 1149780},
      metagraph=[('user', 'book', 'rating')])


In [ ]:
# Weigh the edges by ratings
rating_data = ratings['Book-Rating'].values
ratings_tensor = torch.tensor(rating_data, dtype=torch.float32)
g.edges['rating'].data['rating'] = ratings_tensor 

In [ ]:
# Add age to user feature
ages = users_clean['Age'].values
ages_tensor = torch.tensor(ages, dtype=torch.float32)
g.nodes['user'].data['age'] = ages_tensor

We extract the country from the location by obtaining the expression after the last comma in e.g. nyc, new york, usa

In [437]:
users_clean['Country'] = users_clean['Location'].str.rsplit(',', n=1).str[-1].str.strip()
country_counts = users_clean['Country'].value_counts(normalize=True) 

# We see that less frequent locations do not always contain country names, so we remove values of locations representing less than 1%
rare_countries = country_counts[country_counts < 0.01].index
users_clean.loc[users_clean['Country'].isin(rare_countries), 'Country'] = np.nan

country_ids = {country: idx for idx, country in enumerate(users_clean['Country'].unique())}  # map country to a unique integer
users_clean['CountryId'] = users_clean['Country'].map(country_ids)
countries = users_clean['CountryId'].values
countries_tensor = torch.tensor(countries, dtype=torch.float32)
g.nodes['user'].data['country'] = countries_tensor

/var/folders/qc/8fy08h5s33n4rrmz09_8bv680000gn/T/ipykernel_12836/246620300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_clean['Country'] = users_clean['Location'].str.rsplit(',', n=1).str[-1].str.strip()
/var/folders/qc/8fy08h5s33n4rrmz09_8bv680000gn/T/ipykernel_12836/246620300.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_clean['CountryId'] = users_clean['Country'].map(country_ids)


In [441]:
print(g.nodes['user'])

NodeSpace(data={'age': tensor([18., nan, nan,  ..., 33., 32., nan]), 'country': tensor([0., 0., 1.,  ..., 0., 8., 0.])})
